# 実践 Python 3 輪読会

PyLadies Tokyo 輪読会 #2 (2016-02-03)  
本家ページ：http://www.qtrac.eu/pipbook.html

# 生成に関するデザインパターン

おさらいしましょう！

- 本の中で話されていたオブジェクトの一般的な生成手順って？
  - クラスとは？
  - コンストラクタとは？
  

- 「Pythonic な AbstractFactory」のとこで出てきたテクニック
  - `@classmethod` を使うと，オブジェクトを生成せずにメソッドを使える
  - class の中に class を定義して依存した処理を閉じ込めることで名前空間を綺麗にできる

## 1.2 Builder パターン

- 他のオブジェクトから構成される複雑なオブジェクトを生成するのに利用 ← Abstract Factoryに似てる
- 複雑なオブジェクトを生成するメソッドを提供するだけでなく，複雑なオブジェクト全体の内容も保持する

例によってなんのこっちゃと思うので例を見ていきます

フォームを生成するプログラムについて考える (formbuilder.py)  
ここでは以下の2つの `Builder` を考える

- HTMLを使ったWebフォームを生成する
- PythonとTkinterを用いたGUIフォームを生成する

In [ ]:
def main():
    htmlForm = create_login_form(HtmlFormBuilder())  # HTML形式のフォームを生成
    with open('login.html', "w", encoding="utf-8") as f:  # 生成したフォームを保存
        f.write(htmlForm)
    print("wrote loing.html")

    tkForm = create_login_form(TkFormBuilder())       # Tkinterを利用したフォームを生成
    with open('login.py', "w", encoding="utf-8") as f:  # 生成したフォームを保存
        f.write(tkForm)
    print("wrote login.py")

両方で呼びだされている `create_login_form` って何？

In [ ]:
def create_login_form(builder):
    builder.add_title("Login")
    builder.add_label("Username", 0, 0, target="username")
    builder.add_entry("username", 0, 1)
    builder.add_label("Password", 1, 0, target="password")
    builder.add_entry("password", 1, 1, kind="password")
    builder.add_button("Login", 2, 0)
    builder.add_button("Cancel", 2, 1)
    return builder.form()

`builder` に `HtmlFormBuilder` や `TkFormBuilder` が渡されることで任意のフォームを生成できる  
ここまでは何となく `AbstractFactory` と一緒

__何が違うのか？__  
`HtmlFormBuilder` と `TkFormBuilder` はどちらも `AbstractFormBuilder` という抽象クラスを継承している

------

__Pythonのメタクラスについて__

メタクラスとはクラスの雛形のことである．標準では `type` が利用される  
( ※ `class object` の話はしない，いいね？ )

In [21]:
class Car:
    pass

print(type(Car))

<class 'type'>


In [22]:
print(type(type))

<class 'type'>


`type` をメタクラスとすることで，当たり前に利用している以下の様な機能が使えるようになる

- クラスからインスタンスを生成できる
- クラスの属性へアクセスできる (cls.hogeoge)
- メソッドを定義できる

__Pythonの抽象クラスについて__

Pythonの抽象クラスの機能は， `abc` というモジュールによって提供される．  
抽象クラスを生成するために，メタクラスを `abc.ABCMeta` に変更する．

In [109]:
from abc import ABCMeta, abstractmethod

class Car(metaclass=abc.ABCMeta):
    
    def stop(self):
        print('駐車中')

In [110]:
# メタクラスが abc.ABCMeta に変更されていることを確認
print(type(Car))

<class 'abc.ABCMeta'>


In [111]:
# 実はこのままだとインスタンスを作れてしまう
car = Car()

print(car)
car.stop()

駐車中


In [102]:
class Car(metaclass=abc.ABCMeta):

    def stop(self):
        print('駐車中')

    @abstractmethod
    def drive(self):
        pass

In [103]:
# abstractmethod が定義されていると怒られる
car = Car()

TypeError: Can't instantiate abstract class Car with abstract methods drive

In [104]:
class MyCar(Car):
    pass

# abstractmethod を実装していないので怒られる
car = MyCar()

TypeError: Can't instantiate abstract class MyCar with abstract methods drive

In [105]:
class MyCar(Car):

    # abstractmethodを実装！
    def drive(self):
        print('海に行くよ')

# 今度は怒られない
car = MyCar()

In [107]:
# 実装したメソッドをよんでみる
car.stop()
car.drive()

駐車中
海に行くよ


------

※ `HtmlFormBuilder` ，`TkFormBuilder` ，`AbstractFormBuilder` の実装については実際にコードを見てみよう  

参考：http://qiita.com/disc99/items/840cf9936687f97a482b  
参考：http://stackoverflow.com/questions/11977279/builder-pattern-equivalent-in-python

抽象基底クラスにするとそのクラスはインスタンス化出来なくなる．    
→ C++やJavaなどで書かれたコードをPythonに移植する際に役立つが，実行時にわずかなオーバーヘッドが発生する  
→ 多くのPythonプログラマは単にドキュメントで指示を与えるなどの簡単な方法で済ますことが多い

------

__シーケンスのアンパック ／ ディクショナリのアンパック__

シーケンス，またはディクショナリ中の要素を個別にすべて取り出す処理

In [56]:
sequence = [1, 2, 3, 4, 5, 6]
a, b, *rest = sequence

print(a)
print(b)
print(rest)

1
2
[3, 4, 5, 6]


関数呼び出しで使われるケースが多い

In [115]:
def hoge(a, b, c, d, e):
    print('a: {a}, b: {b}, c: {c}, d: {d}, e: {e}'.format(a=a, b=b, c=c, d=d, e=e))
    
args = (1, 2)
kwargs = {'c': 3, 'd': 4, 'e': 5}

hoge(*args, **kwargs)         # こんな感じで渡せたり
hoge(1, 2, c=3, d=4, e=5)  # これと同じ

a: 1, b: 2, c: 3, d: 4, e: 5
a: 1, b: 2, c: 3, d: 4, e: 5


In [116]:
def hoge(a, b, c, d, e):
    print('a: {a}, b: {b}, c: {c}, d: {d}, e: {e}'.format(a=a, b=b, c=c, d=d, e=e))
    
args = (1, 2)
kwargs = {'c': 3, 'd': 4}

hoge(*args, **kwargs)  # 足りないとちゃんと怒られる

TypeError: hoge() missing 1 required positional argument: 'e'

In [117]:
def hoge(a, b, c, d, e):
    print(locals())                                                                                # ローカルな名前空間の情報
    print('a: {a}, b: {b}, c: {c}, d: {d}, e: {e}'.format(**locals()))  # こんな感じで書き直せたり

args = (1, 2)
kwargs = {'c': 3, 'd': 4, 'e': 5}
    
hoge(*args, **kwargs)

{'b': 2, 'a': 1, 'c': 3, 'e': 5, 'd': 4}
a: 1, b: 2, c: 3, d: 4, e: 5


やり過ぎるとスーパー分かりづらくなるので適度に！

------